In [1]:
#import dependecies
import time
from IPython.display import clear_output
import pandas as pd
from datetime import datetime
from config import weather_api_key
from config import g_key
from citipy import citipy
import numpy as np
import requests


In [2]:
#create random latitudes and longitudes
lats = np.random.uniform(low=-90.000,high= 90.000, size=2000)
lngs = np.random.uniform(low=-180.000,high=180.000, size=2000)

In [3]:
#list comprehension used with citipy module to iterate through random lats & lngs to create a list of tuples containing the city name and the country name
city_names_country_code=[(citipy.nearest_city(i,v).city_name,citipy.nearest_city(i,v).country_code) for i,v in zip(lats,lngs)]

In [4]:
 #for i,v in city_names_country_code:print(i,v)

In [5]:
#query API (start query)
print("Beginning Data Retrieval")
print("-"*30)

#accumulator to build Pandas DataFrame
world_data=[]

#index for record count / percentage
index=0
counter=0
#iterate through list of tuples containing city_names (i) and country_code (v) 
for i,v in city_names_country_code:
    
    #keep track of number of loops with index
    index+=1
    
    #create a dynamic url that queries based on city_name (i)
    url="https://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID="+weather_api_key+"&q="+i
    
    #use the request library to request the data for the current url
    response=requests.get(url)
    
    #convert the reponse into a json object/dictionary
    json_dict=response.json()
    
    #utilize try/except to check the currect json object for our wanted keys and values, saved to a variable without throwing an error if key does not exist in object
    try:
        city_lat = json_dict["coord"]["lat"]
        city_lng = json_dict["coord"]["lon"]
        city_max_temp = json_dict["main"]["temp_max"]
        city_humidity = json_dict["main"]["humidity"]
        city_clouds = json_dict["clouds"]["all"]
        city_wind = json_dict["wind"]["speed"]
        weather_description = json_dict["weather"][0]["description"]
        city_country = json_dict["sys"]["country"]
        
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(json_dict["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information into world_data list.
        world_data.append({"City": i,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max_Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind_Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                          "Weather_Description":weather_description})
        
        #After the data is succesfully collected print the following message: city, country, iteration (index), and percentage (index/len(city_names_country_code))
        print(f'city:    {i.title()}\ncountry: {v.upper()}\n\t{response} {index} of {len(city_names_country_code)} | {"{:.2f}".format((index/len(city_names_country_code)*100))}% completed\n')
        counter+=1
        if counter == 10:
            clear_output()
            counter=0
    #KeyError would reflect that the json object does not contain the information we are looking for. we catch and skip to the next object
    except KeyError:
        print(f"\n\nWeather data for City {i.title()} in Country {v.upper()} not found. Skipping...\n\n")
        counter+=1
        if counter == 10:
            clear_output()
            counter=0
        pass
        
print("-"*30)
print("Data Retrieval Complete      ")
print("-"*30)


Beginning Data Retrieval
------------------------------
city:    Cape Town
country: ZA
	<Response [200]> 1 of 2000 | 0.05% completed

city:    Kant
country: KG
	<Response [200]> 2 of 2000 | 0.10% completed

city:    Pacific Grove
country: US
	<Response [200]> 3 of 2000 | 0.15% completed

city:    Hilo
country: US
	<Response [200]> 4 of 2000 | 0.20% completed

city:    Tura
country: RU
	<Response [200]> 5 of 2000 | 0.25% completed

city:    Naze
country: JP
	<Response [200]> 6 of 2000 | 0.30% completed

city:    Ugoofaaru
country: MV
	<Response [200]> 7 of 2000 | 0.35% completed

city:    Emerald
country: AU
	<Response [200]> 8 of 2000 | 0.40% completed

city:    Camiri
country: BO
	<Response [200]> 9 of 2000 | 0.45% completed



Weather data for City Barentsburg in Country SJ not found. Skipping...


city:    Singarayakonda
country: IN
	<Response [200]> 11 of 2000 | 0.55% completed

city:    Manokwari
country: ID
	<Response [200]> 12 of 2000 | 0.60% completed

city:    Qaanaaq
country:

city:    Vaini
country: TO
	<Response [200]> 107 of 2000 | 5.35% completed

city:    La Ronge
country: CA
	<Response [200]> 108 of 2000 | 5.40% completed

city:    Fortuna
country: US
	<Response [200]> 109 of 2000 | 5.45% completed

city:    Saldanha
country: ZA
	<Response [200]> 110 of 2000 | 5.50% completed

city:    Ushuaia
country: AR
	<Response [200]> 111 of 2000 | 5.55% completed

city:    Coihaique
country: CL
	<Response [200]> 112 of 2000 | 5.60% completed



Weather data for City Karkaralinsk in Country KZ not found. Skipping...


city:    Katsuura
country: JP
	<Response [200]> 114 of 2000 | 5.70% completed

city:    Puerto Del Rosario
country: ES
	<Response [200]> 115 of 2000 | 5.75% completed

city:    Awbari
country: LY
	<Response [200]> 116 of 2000 | 5.80% completed

city:    Abomey
country: BJ
	<Response [200]> 117 of 2000 | 5.85% completed

city:    Puerto Ayora
country: EC
	<Response [200]> 118 of 2000 | 5.90% completed

city:    Ushuaia
country: AR
	<Response [200]> 11

city:    Hilo
country: US
	<Response [200]> 211 of 2000 | 10.55% completed

city:    Ushuaia
country: AR
	<Response [200]> 212 of 2000 | 10.60% completed

city:    Kapaa
country: US
	<Response [200]> 213 of 2000 | 10.65% completed

city:    Krasnoselkup
country: RU
	<Response [200]> 214 of 2000 | 10.70% completed

city:    Vila Velha
country: BR
	<Response [200]> 215 of 2000 | 10.75% completed

city:    Tuatapere
country: NZ
	<Response [200]> 216 of 2000 | 10.80% completed

city:    Rikitea
country: PF
	<Response [200]> 217 of 2000 | 10.85% completed

city:    Longyearbyen
country: SJ
	<Response [200]> 218 of 2000 | 10.90% completed

city:    Jamestown
country: SH
	<Response [200]> 219 of 2000 | 10.95% completed

city:    Valparaiso
country: CL
	<Response [200]> 220 of 2000 | 11.00% completed



Weather data for City Taolanaro in Country MG not found. Skipping...




Weather data for City Attawapiskat in Country CA not found. Skipping...


city:    Priekule
country: LT
	<Response [200]

city:    Chokurdakh
country: RU
	<Response [200]> 314 of 2000 | 15.70% completed

city:    Haines Junction
country: CA
	<Response [200]> 315 of 2000 | 15.75% completed

city:    Kirakira
country: SB
	<Response [200]> 316 of 2000 | 15.80% completed

city:    Tiarei
country: PF
	<Response [200]> 317 of 2000 | 15.85% completed

city:    Barrow
country: US
	<Response [200]> 318 of 2000 | 15.90% completed

city:    Pervomayskiy
country: RU
	<Response [200]> 319 of 2000 | 15.95% completed

city:    Frontignan
country: FR
	<Response [200]> 320 of 2000 | 16.00% completed

city:    Lebu
country: CL
	<Response [200]> 321 of 2000 | 16.05% completed

city:    Dingle
country: IE
	<Response [200]> 322 of 2000 | 16.10% completed

city:    Tuatapere
country: NZ
	<Response [200]> 323 of 2000 | 16.15% completed

city:    Qaanaaq
country: GL
	<Response [200]> 324 of 2000 | 16.20% completed



Weather data for City Illoqqortoormiut in Country GL not found. Skipping...


city:    Barrow
country: US
	<Respo



Weather data for City Malwan in Country IN not found. Skipping...


city:    Fort Nelson
country: CA
	<Response [200]> 418 of 2000 | 20.90% completed

city:    Avarua
country: CK
	<Response [200]> 419 of 2000 | 20.95% completed

city:    Tailai
country: CN
	<Response [200]> 420 of 2000 | 21.00% completed

city:    Bredasdorp
country: ZA
	<Response [200]> 421 of 2000 | 21.05% completed



Weather data for City Barentsburg in Country SJ not found. Skipping...


city:    Vila Franca Do Campo
country: PT
	<Response [200]> 423 of 2000 | 21.15% completed

city:    Hobart
country: AU
	<Response [200]> 424 of 2000 | 21.20% completed

city:    Tasiilaq
country: GL
	<Response [200]> 425 of 2000 | 21.25% completed

city:    Gari
country: RU
	<Response [200]> 426 of 2000 | 21.30% completed

city:    Mar Del Plata
country: AR
	<Response [200]> 427 of 2000 | 21.35% completed

city:    Albany
country: AU
	<Response [200]> 428 of 2000 | 21.40% completed

city:    Arraial Do Cabo
country: BR
	<Respon

...

city:    Mataura
country: PF
	<Response [200]> 1547 of 2000 | 77.35% completed



Weather data for City Taolanaro in Country MG not found. Skipping...


city:    Carnarvon
country: AU
	<Response [200]> 1549 of 2000 | 77.45% completed

city:    Thompson
country: CA
	<Response [200]> 1550 of 2000 | 77.50% completed

city:    Talnakh
country: RU
	<Response [200]> 1551 of 2000 | 77.55% completed

city:    Albany
country: AU
	<Response [200]> 1552 of 2000 | 77.60% completed

city:    Tuatapere
country: NZ
	<Response [200]> 1553 of 2000 | 77.65% completed

city:    Fortuna
country: US
	<Response [200]> 1554 of 2000 | 77.70% completed

city:    Barrow
country: US
	<Response [200]> 1555 of 2000 | 77.75% completed

city:    Hilo
country: US
	<Response [200]> 1556 of 2000 | 77.80% completed

city:    Albany
country: AU
	<Response [200]> 1557 of 2000 | 77.85% completed

city:    Pyapon
country: MM
	<Response [200]> 1558 of 2000 | 77.90% completed

city:    Novokayakent
country: RU
	<Response [200

city:    Tocopilla
country: CL
	<Response [200]> 1649 of 2000 | 82.45% completed

city:    Albany
country: AU
	<Response [200]> 1650 of 2000 | 82.50% completed

city:    Pilar
country: PH
	<Response [200]> 1651 of 2000 | 82.55% completed

city:    Ushuaia
country: AR
	<Response [200]> 1652 of 2000 | 82.60% completed

city:    Dingle
country: IE
	<Response [200]> 1653 of 2000 | 82.65% completed

city:    Changji
country: CN
	<Response [200]> 1654 of 2000 | 82.70% completed

city:    Yellowknife
country: CA
	<Response [200]> 1655 of 2000 | 82.75% completed

city:    Ushuaia
country: AR
	<Response [200]> 1656 of 2000 | 82.80% completed

city:    Takhtamygda
country: RU
	<Response [200]> 1657 of 2000 | 82.85% completed

city:    Aklavik
country: CA
	<Response [200]> 1658 of 2000 | 82.90% completed

city:    Praia Da Vitoria
country: PT
	<Response [200]> 1659 of 2000 | 82.95% completed

city:    Bluff
country: NZ
	<Response [200]> 1660 of 2000 | 83.00% completed

city:    Waipawa
country: N

city:    Tuktoyaktuk
country: CA
	<Response [200]> 1750 of 2000 | 87.50% completed



Weather data for City Taolanaro in Country MG not found. Skipping...


city:    Raudeberg
country: NO
	<Response [200]> 1752 of 2000 | 87.60% completed

city:    Sola
country: VU
	<Response [200]> 1753 of 2000 | 87.65% completed

city:    Adzope
country: CI
	<Response [200]> 1754 of 2000 | 87.70% completed

city:    Hilo
country: US
	<Response [200]> 1755 of 2000 | 87.75% completed

city:    Atambua
country: ID
	<Response [200]> 1756 of 2000 | 87.80% completed

city:    Carbonia
country: IT
	<Response [200]> 1757 of 2000 | 87.85% completed

city:    Hermanus
country: ZA
	<Response [200]> 1758 of 2000 | 87.90% completed



Weather data for City Marcona in Country PE not found. Skipping...


city:    Vaini
country: TO
	<Response [200]> 1760 of 2000 | 88.00% completed

city:    Hobart
country: AU
	<Response [200]> 1761 of 2000 | 88.05% completed



Weather data for City Barentsburg in Country SJ not foun

city:    Saint Augustine
country: US
	<Response [200]> 1852 of 2000 | 92.60% completed

city:    Barrow
country: US
	<Response [200]> 1853 of 2000 | 92.65% completed

city:    Leningradskiy
country: RU
	<Response [200]> 1854 of 2000 | 92.70% completed

city:    Rikitea
country: PF
	<Response [200]> 1855 of 2000 | 92.75% completed

city:    Berdigestyakh
country: RU
	<Response [200]> 1856 of 2000 | 92.80% completed

city:    Ushuaia
country: AR
	<Response [200]> 1857 of 2000 | 92.85% completed

city:    Los Llanos De Aridane
country: ES
	<Response [200]> 1858 of 2000 | 92.90% completed

city:    Rikitea
country: PF
	<Response [200]> 1859 of 2000 | 92.95% completed

city:    Zavodoukovsk
country: RU
	<Response [200]> 1860 of 2000 | 93.00% completed

city:    Arandis
country: NA
	<Response [200]> 1861 of 2000 | 93.05% completed

city:    Chokurdakh
country: RU
	<Response [200]> 1862 of 2000 | 93.10% completed



Weather data for City Rungata in Country KI not found. Skipping...




Weathe

city:    Blackwater
country: AU
	<Response [200]> 1954 of 2000 | 97.70% completed

city:    Mahebourg
country: MU
	<Response [200]> 1955 of 2000 | 97.75% completed

city:    Mabaruma
country: GY
	<Response [200]> 1956 of 2000 | 97.80% completed

city:    Gurupi
country: BR
	<Response [200]> 1957 of 2000 | 97.85% completed

city:    Leningradskiy
country: RU
	<Response [200]> 1958 of 2000 | 97.90% completed



Weather data for City Illoqqortoormiut in Country GL not found. Skipping...


city:    San Cristobal
country: EC
	<Response [200]> 1960 of 2000 | 98.00% completed

city:    Leh
country: IN
	<Response [200]> 1961 of 2000 | 98.05% completed

city:    Faya
country: TD
	<Response [200]> 1962 of 2000 | 98.10% completed



Weather data for City Meyungs in Country PW not found. Skipping...




Weather data for City Barentsburg in Country SJ not found. Skipping...


city:    Albany
country: AU
	<Response [200]> 1965 of 2000 | 98.25% completed

city:    Butaritari
country: KI
	<Response [2

In [6]:
#create a pandas DataFrame from the list of dictionarys/ array of json objects
world_temp_data_by_city = pd.DataFrame(world_data)
#display part of collected data in the DataFrame
world_temp_data_by_city.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Weather_Description
0,cape town,-33.93,18.42,64.00,93,0,9.17,ZA,2020-10-23 02:42:02,clear sky
1,kant,27.80,79.80,77.31,36,0,4.54,IN,2020-10-23 02:45:11,clear sky
2,pacific grove,36.62,-121.92,61.00,82,1,3.36,US,2020-10-23 02:45:11,clear sky
3,hilo,19.73,-155.09,84.20,66,1,14.99,US,2020-10-23 02:42:17,clear sky
4,tura,25.52,90.22,75.60,88,47,1.66,IN,2020-10-23 02:45:12,scattered clouds


In [7]:
#export Data to a csv
world_temp_data_by_city.to_csv("WeatherPy_Database.csv")
